# Lab 1: EDA

In [25]:
import pandas as pd
import numpy as np
import pingouin as pg
from sklearn.datasets import fetch_california_housing

In [26]:
# Carga el dataset de California Housing
desc = fetch_california_housing()
X,Y = fetch_california_housing(return_X_y = True)
data = pd.DataFrame(X, columns =desc['feature_names'] )
data[desc['target_names'][0]] = Y 

**Para el conjunto de datos dado, realice la implementacion de la siguiente estrategia de eliminacion de informacion redundante:**

1.	Entre las variables explicativas (todas las variables excepto la variable objetivo), identifique los pares con una correlación de Pearson superior al 80%. Para cada par, elimine la variable que presente una menor correlación parcial con la variable objetivo.
2.	Aplique una estrategia adicional de reducción de redundancia (al conjunto de datos original) evaluando el índice de correlación múltiple entre las variables explicativas, y elimine aquellas cuya variabilidad explicada sea superior al 80% en funcion de las demas.

In [ ]:
### Ejemplo correlacion parcial:
# data.pcorr()

In [32]:
### TO DO 1: Eliminacion de reduncancia basado en par de variables

### conjunto de datos dividido en variables explicativas y variable objetivo
data_reg = data.copy()
X_reg = data.drop(columns = ['MedHouseVal'])
Y_reg = data['MedHouseVal']

### Encontrar pares de variables con alta correlacion
X_reg_reduced = X_reg.copy()
corr_matrix = X_reg.corr().abs()
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
max_corr = upper_tri.max().max()
print("Correlación máxima entre variables explicativas:", max_corr)
while max_corr > 0.8:
    high_corr_pairs = [(col1, col2) for col1 in upper_tri.columns for col2 in upper_tri.index if upper_tri.loc[col2, col1] > 0.8]
    print("Pares de variables con correlación > 0.8:", high_corr_pairs)
    ### Eliminar variables con menor correlacion parcial con la variable objetivo
    to_drop = set()
    for var1, var2 in [high_corr_pairs[0]]:
        ### correlacion parcial con la variable objetivo
        partial_corr = data_reg.pcorr()['MedHouseVal'].to_dict()
        corr_var1 = partial_corr[var1]
        corr_var2 = partial_corr[var2]
        if abs(corr_var1) < abs(corr_var2):
            to_drop.add(var1)
        else:
            to_drop.add(var2)
    X_reg_reduced = X_reg_reduced.drop(columns = list(to_drop))
    data_reg = data_reg.drop(columns = list(to_drop))
    
    corr_matrix = X_reg_reduced.corr().abs()
    upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    max_corr = upper_tri.max().max()
    print("Correlación máxima entre variables explicativas tras eliminación:", max_corr)
    print(to_drop)

Correlación máxima entre variables explicativas: 0.9246644339150362
Pares de variables con correlación > 0.8: [('AveBedrms', 'AveRooms'), ('Longitude', 'Latitude')]
Correlación máxima entre variables explicativas tras eliminación: 0.9246644339150362
{'AveRooms'}
Pares de variables con correlación > 0.8: [('Longitude', 'Latitude')]
Correlación máxima entre variables explicativas tras eliminación: 0.29624423977353637
{'Longitude'}


In [33]:
### TO DO 2: Eliminacion de reduncancia basado en multiples variables

### Calculo del indice de correlacion multiple
sigma = X_reg.cov()
sigma_inv = np.linalg.inv(sigma)

corr_multiple = 1 - 1/np.diag(sigma_inv)/np.diag(sigma)
print("Índice de correlación múltiple para cada variable explicativa:")
print(corr_multiple)

### ciclo para eliminar variables con alta correlacion multiple
X_reg_reduced = X_reg.copy()
max_corr_multiple = max(corr_multiple)
while max_corr_multiple > 0.8:
    to_drop = np.argmax(corr_multiple)
    print(f"Eliminando variable {X_reg_reduced.columns[to_drop]} con índice de correlación múltiple {max_corr_multiple}")
    X_reg_reduced = X_reg_reduced.drop(columns = [X_reg_reduced.columns[to_drop]])
    
    sigma = X_reg_reduced.cov()
    sigma_inv = np.linalg.inv(sigma)
    corr_multiple = 1 - 1/np.diag(sigma_inv)/np.diag(sigma)
    max_corr_multiple = max(corr_multiple)
    print("Nuevo índice de correlación múltiple máximo tras eliminación:", max_corr_multiple)


Índice de correlación múltiple para cada variable explicativa:
[0.60020701 0.1943632  0.88013596 0.85704064 0.12136195 0.00825572
 0.89244564 0.88842104]
Eliminando variable Latitude con índice de correlación múltiple 0.8924456441475132
Nuevo índice de correlación múltiple máximo tras eliminación: 0.866904134133273
Eliminando variable AveRooms con índice de correlación múltiple 0.866904134133273
Nuevo índice de correlación múltiple máximo tras eliminación: 0.12013044283971674
